# Assignment for

In [3]:
%%bash
wget --no-check-certificate -O "chap4lib.7z" "https://docs.google.com/uc?export=download&id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ"
7z x chap4lib.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 1458 bytes (2 KiB)

Extracting archive: chap4lib.7z
--
Path = chap4lib.7z
Type = 7z
Physical Size = 1458
Headers Size = 130
Method = LZMA2:6k
Solid = -
Blocks = 1


Would you like to replace the existing file:
  Path:     ./chap4lib.py
  Size:     5251 bytes (6 KiB)
  Modified: 2025-05-25 16:46:08
with the file from archive:
  Path:     chap4lib.py
  Size:     5251 bytes (6 KiB)
  Modified: 2025-05-25 16:46:08
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? 

--2025-06-04 06:31:02--  https://docs.google.com/uc?export=download&id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ
Resolving docs.google.com (docs.google.com)... 173.194.206.139, 173.194.206.138, 173.194.206.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.206.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ&export=download [following]
--2025-06-04 06:31:02--  https://drive.usercontent.google.com/download?id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.193.132, 2607:f8b0:4001:c0f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.193.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1458 (1.4K) [application/octet-stream]
Saving to: ‘chap4lib.7z’

     0K .                                                     100% 14

CalledProcessError: Command 'b'wget --no-check-certificate -O "chap4lib.7z" "https://docs.google.com/uc?export=download&id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ"\n7z x chap4lib.7z\n'' returned non-zero exit status 134.

In [9]:
import typing
T = typing.TypeVar('T')


In [10]:
from chap4lib import *
from typing import Callable, Any

SyntaxError: invalid syntax (chap4lib.py, line 7)

## A1.

### Background

In our lectures (refer to `chap4.pdf`), we introduced the language $Lang_3$, which is a kernel imperative language. We discussed its syntax for expressions ($Exp_3$), definitions ($Def_3$), and commands ($Lang_3$). We also explored how to represent these constructs using Python classes and algebraic interfaces, as provided in `chap4lib.py`.

The key components are:

  * `Exp3Alg[ExpT]`: An algebra for building expressions. Its methods return objects of type `ExpT`.
  * `Def3Alg[ExpT, CmdT, DefT]`: An algebra for building definitions. Its methods (like `let` and `proc`) take expressions (of type `ExpT`) and commands (of type `CmdT`) and return objects of type `DefT`.
  * `Lang3Alg[ExpT, DefT, CmdT]`: An algebra for building commands. Its methods (like `assign`, `skip`, `seq`, `cond`, `loop`, `call_...`, `block`) take expressions (`ExpT`) and definitions (`DefT`) and return objects of type `CmdT`.

### Your Task

You are to implement Python functions with the given signatures. These functions will represent the given $Lang_3$ programs that define and calls a factorial procedure.




### A1-1.

**The $Lang_3$ Program to Implement:**

The program consists of an outer block that defines a procedure `factorial` and then calls it. The procedure itself uses an inner block and a `while` loop to compute the factorial.

```text
begin
  proc factorial(n) {             // procedure
    begin                         // block
      let y = ↑(1);               // definition
      while (not(↓n == 1)) do {   // command
        y := ↓y * ↓n;
        n := ↓n - 1
      }
    end
  }
  call factorial(↑3)              // Use lang.call_const() for call by value, constant
end
```

In [1]:
def call_fact_pgm_const[ExpT, DefT, CmdT](
        exp: Exp3Alg[ExpT],
        de: Def3Alg[ExpT, CmdT, DefT],
        lang: Lang3Alg[ExpT, DefT, CmdT]
    ) -> CmdT:
    factorial_def = de.proc(
        "factorial", "n",
        lang.block(
        [de.let("y", exp.alloc(exp.cint(1)))],  # 定義リストのみをリスト内に
        lang.loop(  # 第2引数: コマンドを単独で渡す
            exp.not_(exp.eq(exp.deref(exp.var("n")), exp.cint(1))),
            lang.seq(
                lang.assign("y", exp.mult(exp.deref(exp.var("y")), exp.deref(exp.var("n")))),
                lang.assign("n", exp.minus(exp.deref(exp.var("n")), exp.cint(1)))
                )
            )
        )
    )

    return lang.block(
        [factorial_def],
        lang.call_const("factorial", exp.alloc(exp.cint(3)))  # Correct format
    )

SyntaxError: expected '(' (<ipython-input-1-b57b12171e25>, line 1)

### A1-2.

**The $Lang_3$ Program to Implement:**

The program consists of an outer block that defines a procedure `factorial` and then calls it. The procedure parameter `n` is handled using call-by-value, mutable semantics. This means `n` behaves as a local, mutable variable within `factorial`, initialized with the value passed in the call.

```text
begin
  proc factorial(n) {             // procedure
    begin                         // block
      let y = ↑(1);               // definition
      while (not(↓n == 1)) do {   // command
        y := ↓y * ↓n;
        n := ↓n - 1
      }
    end
  }
  call factorial(3)               // Use lang.call_mutable() for call by value, mutable
end
```

In [ ]:
def call_fact_pgm_mutable[ExpT, DefT, CmdT](
    exp: Exp3Alg[ExpT],
    de: Def3Alg[ExpT, CmdT, DefT],
    lang: Lang3Alg[ExpT, DefT, CmdT]) -> CmdT:
    factorial_def = de.proc(
        "factorial","n",
        lang.block(
            [lang.let("y",exp.alloc(exp.cint(1)))],
            lang.loop(
                exp.not_(exp.eq(exp.deref(exp.var("n")),exp.cint(1))),
                lang.seq(
                    lang.assign("y",exp.mult(exp.deref(exp.var("y")),exp.deref(exp.var("n")))),
                    lang.assign("n",exp.minus(exp.deref(exp.var("n")),exp.cint(1)))
                    )
                )
            )
        )
    return lang.block(
        [factorial_def],
        lang.call_const("factorial", exp.alloc(exp.cint(3)))  # Correct format
    )


### A1-3.

**The $Lang_3$ Program to Implement:**

The program consists of an outer block that defines the `factorial` procedure and a variable `x`. It then calls `factorial` passing `x` by reference.

```text
begin
  proc factorial(n) {             // procedure
    begin                         // block
      let y = ↑(1);               // definition
      while (not(↓n == 1)) do {   // command
        y := ↓y * ↓n;
        n := ↓n - 1
      }
    end
  }
  let x = ↑3;
  call p(x)                       // Use lang.call_ref() for call by reference
end
```

In [ ]:
def call_fact_pgm_ref[ExpT, DefT, CmdT](
    exp: Exp3Alg[ExpT],
    de: Def3Alg[ExpT, CmdT, DefT],
    lang: Lang3Alg[ExpT, DefT, CmdT]
) -> CmdT:
    # --- YOUR IMPLEMENTATION GOES HERE ---
    pass

## A2.

This assignment requires you to implement key components of an interpreter for $Lang_3$, an imperative language with explicit memory management, definitions, and procedures. You will be working with an algebraic approach, implementing Python classes that define the semantics for expressions ($Exp_3$), definitions ($Def_3$), and the language commands ($Lang_3$) themselves.

The primary objectives are:
* To understand and implement the evaluation semantics for expressions, including those with side effects (memory allocation).
* To implement the semantics for definitions, including procedure definitions with static (lexical) scoping.
* To implement the big-step operational semantics for language commands, including assignments, control flow, blocks, and procedure calls (call-by-value constant, call-by-value mutable, and call-by-reference) using static scope.
* To gain practical experience with functional approaches to interpreter writing in Python, using higher-order functions to represent computations.

You **must** use the interfaces and data structures provided in `chap4lib.py`.

### Core Types for Evaluation Functions

Your implementations will revolve around functions that take the current state `(EnvType, MemType)` and return new states or values. The key evaluation function types (as seen in the slides [cite: 34, 46, 81]) are:

* `EvalResultType = Callable[[StateType], tuple[Value, MemType]]`: For evaluating expressions in $Exp_3$. It takes a state and returns the resulting `Value` and the (potentially modified) `MemType`.
* `EvalDefResultType = Callable[[StateType], StateType]`: For evaluating definitions in $Def_3$. It takes a state and returns a new state `(EnvType, MemType)` reflecting the new bindings.
* `EvalCmdResultType = Callable[[StateType], MemType]`: For executing commands in $Lang_3$. It takes a state `(EnvType, MemType)` and returns the final `MemType` after the command's execution.

### Tasks

You need to define and implement the following three Python classes:


#### Task 1: Implement `Exp3Eval`

This class implements the `Exp3Alg` interface and defines the evaluation semantics for $Exp_3$ expressions.

#### Task 2: Implement `Def3Eval`

This class implements the `Def3Alg` interface and defines the semantics for $Def_3$ definitions.

**Hint for `proc` closure:** A procedure value (closure for static scope) should store `(parameter_name, body_command, definition_environment)`. You can define a simple class `VProc(Value)` to encapsulate this tuple or store the tuple directly in the environment. Ensure that this value can be retrieved and correctly interpreted during procedure calls.

#### Task 3: Implement `Lang3EvalSS` (Static Scoping)

This class implements the `Lang3Alg` interface and defines the big-step operational semantics for $Lang_3$ commands, incorporating **static scoping** for procedures.


## A3.

This assignment builds upon your understanding of $Lang_3$ by requiring you to implement its **small-step operational semantics**. Unlike big-step semantics which define the overall result of a command, small-step semantics describe how commands execute one step at a time. This approach is crucial for understanding more complex language features like concurrency and for a finer-grained view of program execution.

You will define Python classes to represent $Lang_3$ commands. Each command class will encapsulate its own single-step execution logic via a `step` method. This approach uses polymorphism to avoid large conditional statements for different command types.

You will focus on the core commands: assignment, skip, sequence, conditional, and loop. **Procedure calls and blocks are excluded from this assignment** as their small-step semantics were not covered in detail in the lecture slides.

The primary objectives are:
* To understand and implement small-step operational semantics.
* To represent program syntax as data structures (command objects) with associated behavior (step methods).
* To implement state transition logic within each command class based on formal semantic rules.

### Core Concepts for Small-Step Semantics

* **Configuration:** A configuration is a pair `⟨c, Mem⟩`, where `c` is the command (or part of a command) remaining to be executed, and `Mem` is the current memory state.
* **Transition Relation (`↪_Env`):** Describes a single step of computation: `⟨c₁, Mem₁⟩ ↪_Env ⟨c₂, Mem₂⟩`.
* **Terminal Configuration:** Execution stops when a command `c` in state `(Env, Mem)` has no further steps. For this assignment, a command sequence terminates when it reduces to a single `skip` command in a configuration `⟨skip, Mem⟩`.

### 4. Tasks

#### Step 1: Command Representation (`SmallStepCmd`) with `step` Methods

First, you need to define Python classes to represent the $Lang_3$ commands. Each class will now also include a `step` method that defines how an instance of that command takes a single execution step.

Define the following classes. We'll use `SmallStepCmdType` as a type hint for these command objects. You should also define your `Exp3Eval` class from the previous assignment or ensure it's available, as its `EvalResultType` is needed.

```python
class SmallStepCmd: # base class
    def step(self,
             env: EnvType,
             current_mem: MemType,
             exp_eval_instance: Exp3Eval # Use forward reference or import
            ) -> Tuple[SmallStepCmd, MemType]:
        """Abstract method for performing one step of execution."""
        raise NotImplementedError
    # Consider adding __repr__ for debugging all command classes

class AssignCmd(SmallStepCmd):
    def __init__(self, var_name: str, expr_eval_func: 'EvalResultType'):
        self.var_name = var_name
        self.expr_eval_func = expr_eval_func

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # Implements Rule S1 (see Part 3 for details)
        raise NotImplementedError # To be implemented by student

class SkipCmd(SmallStepCmd):
    def __init__(self):
        pass # No specific data for SkipCmd

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # A SkipCmd is already in a "reduced" form.
        # It doesn't take a further step to a different command.
        # The interpreter loop (Part 4) will identify SkipCmd as a terminal state.
        return (self, current_mem)

class SeqCmd(SmallStepCmd):
    def __init__(self, cmd1: SmallStepCmd, cmd2: SmallStepCmd):
        self.cmd1 = cmd1
        self.cmd2 = cmd2

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # Implements Rules S2 and S3 (see Part 3 for details)
        raise NotImplementedError # To be implemented by student

class CondCmd(SmallStepCmd):
    def __init__(self, cond_expr_eval_func: 'EvalResultType', then_cmd: SmallStepCmd, else_cmd: SmallStepCmd):
        self.cond_expr_eval_func = cond_expr_eval_func
        self.then_cmd = then_cmd
        self.else_cmd = else_cmd

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # Implements Rules S4 and S5 (see Part 3 for details)
        raise NotImplementedError # To be implemented by student

class LoopCmd(SmallStepCmd):
    def __init__(self, cond_expr_eval_func: 'EvalResultType', body_cmd: SmallStepCmd):
        self.cond_expr_eval_func = cond_expr_eval_func
        self.body_cmd = body_cmd

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # Implements Rules S6 and S7 (see Part 3 for details)
        raise NotImplementedError # To be implemented by student
```
*(Ensure `EvalResultType` and `Exp3Eval` are correctly defined and imported based on your project structure. `chap4lib.py` provides `EnvType`, `MemType`, `Value`. Your `Exp3Eval` solution from the previous assignment provides `EvalResultType`.)*

#### Step 2: Implementing `Lang3EvalSmall`

Next, implement the `Lang3EvalSmall` class. This class will implement the `Lang3Alg` interface. Its methods will **construct and return instances of your command representation classes** from Part 1.

* `Lang3EvalSmall` should implement `Lang3Alg[EvalResultType, typing.Any, SmallStepCmdType]`.
    * `ExpT` is `EvalResultType`.
    * `DefT` can be `typing.Any` as `block` (which uses definitions) is not part of this assignment's small-step implementation.
    * `T` (the return type of command construction methods) will be `SmallStepCmdType`.

**Implementation Details:**
* `assign(self, x: str, e: EvalResultType) -> SmallStepCmdType`: Returns an `AssignCmd(x, e)` instance.
* `skip(self) -> SmallStepCmdType`: Returns a `SkipCmd()` instance.
* `seq(self, c1: SmallStepCmdType, c2: SmallStepCmdType) -> SmallStepCmdType`: Returns a `SeqCmd(c1, c2)` instance.
* `cond(self, e: EvalResultType, c1: SmallStepCmdType, c2: SmallStepCmdType) -> SmallStepCmdType`: Returns a `CondCmd(e, c1, c2)` instance.
* `loop(self, e: EvalResultType, c: SmallStepCmdType) -> SmallStepCmdType`: Returns a `LoopCmd(e, c)` instance.
* **Excluded Methods:** The methods `block`, `call_const`, `call_mutable`, and `call_ref` **should raise `NotImplementedError`**.

#### Step 3: Implementing `step` Methods in Command Classes

This is the core of the small-step interpreter. Each command class you defined in Part 1 (`AssignCmd`, `SeqCmd`, `CondCmd`, `LoopCmd`) must implement its own `step` method. The `SkipCmd.step` method is trivial as it represents a completed step.

**Method Signature (to be implemented in each relevant command class):**
```python
# Inside each of your command classes (AssignCmd, SeqCmd, CondCmd, LoopCmd)
def step(self,
         env: EnvType,
         current_mem: MemType,
         exp_eval_instance: Exp3Eval # Your working Exp3Eval instance
        ) -> Tuple[SmallStepCmd, MemType]: # Return type uses forward reference
    # ... implementation specific to this command type based on S1-S7 ...
```